# Data Visualization Using Chloropleth Maps

## Example Application: 2020 Primary Vote Margin in TX31

In [1]:
"""
Use a chloropleth to visualize the distribution of precincts that went for or against Donna Imam in the March 2020
Democratic Primary. Vote Margin is (Donna vote %) - (runoff opponent %). See vandata/ for the input data file.
"""

import geopandas as gpd
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.transform import factor_cmap
from bokeh.palettes import brewer, Spectral6
import json
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column
from bokeh.io import export_png
from precinctmapping import Chloropleth

#This shapefile contains all Texas election precincts, not just TX31
shapefile = 'texasprecincts/Precincts.shp'
# Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)
gdf = gdf[gdf['CNTY'].isin(['27', '491'])] #Filters for Bell and Williamson Counties, which make up TX31

#read in the data file and take just the columns of interest
datafile = pd.read_csv('vandata/2020pvotesbyprecinct.csv')
datafile['Margin'] = datafile['Margin']*100
datafile = datafile[['Precinct','Margin']]

#I think at one point the precinct column was giving me trouble so I put this here, it may or may not be necessary
for i in range(len(datafile)):
    datafile['Precinct'].iloc[i] = str(datafile['Precinct'].iloc[i])
    if len(datafile['Precinct'].iloc[i]) == 3:
        datafile['Precinct'].iloc[i] = '0' + datafile['Precinct'].iloc[i]
print(datafile.head())

#merge geodata and data, convert to json
merged = gdf.merge(datafile, 'right', left_on='PREC', right_on='Precinct')
#print(merged.head)
merged_json = json.loads(merged.to_json())
jsondata = json.dumps(merged_json)
mapscale = (datafile['Margin'].min(),datafile['Margin'].max())
print(mapscale)

#Instantiate Chloropleth object and create map
my_map = Chloropleth(json_data = jsondata, 
                     varname = 'Margin',
                     figtitle = 'TX31 Vote Margin by Precinct', 
                     palette = brewer['Spectral'][5][::-1],
                     mapscale = (datafile['Margin'].min(),datafile['Margin'].max()))
my_map.create()

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


  Precinct     Margin
0     0101 -10.825688
1     0102   8.203125
2     0103  -1.449275
3     0104  -7.692308
4     0105   1.176471
(-31.780000000000001, 13.043478259999999)


Loading BokehJS ...